In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_graded_features(audio_features, df_media):
    TEMP = df_media[['doc','moyenne']].groupby(['doc']).agg(['count','mean']).droplevel(0,axis=1)
    RES = pd.merge(audio_features[selected_columns], TEMP, left_on='SCENE', right_on='doc').sort_values(by=['SCENE'])
    RES = RES.reset_index(drop=True)
    
    # remplacement des NaN par les moyennes
    for c in selected_features:
        RES.loc[np.isnan(RES.loc[:,c]),c] = np.nanmean(RES.loc[:,c])
    
    return RES

#### chargement des données et des features

In [3]:
%run loading_data.ipynb

graded = annot[(annot['isGraded'] == 1)].copy()
graded['moyenne'] = graded[noteurs].apply(np.nanmean, axis=1)

selected_features = ['SR','SNR','CONF','SCORE']
selected_columns = ['SCENE'] + selected_features
selected_y = 'mean'

audio_features = pd.read_csv('./data/audio_features.csv', sep='§', engine='python', index_col=0, encoding='utf-8')
audio_features.head(2)

audio : 55
audio_only : 33
video : 44
texte : 55
texte_only : 33


,SCENE,SR,SNR,CONF,RECON,SCORE,XML
0,100_1,0.634000,0.952384,0.889849,Bonsoir intérêt pour coiffeur c'est pas sérieu...,0.573276,Bonsoir Tu veux pas te garer Pour quoi faire M...
1,107_7,0.944506,0.998807,0.882336,pourquoi trouver la je vous en prie je vous in...,0.943565,Qu est ce qui se passe Vous en faites un bouca...


## Régression linéaire, leave on out

In [4]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression

DF = get_graded_features(audio_features, audio_only)

X = DF[selected_features].values
y = DF[selected_y].values
preds_test = np.zeros(len(X))

loo = LeaveOneOut()
loo.get_n_splits(X)

for i,(train_index, test_index) in enumerate(loo.split(X)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    reg = LinearRegression().fit(X_train, y_train)
    preds_test[i] = reg.predict(X_test)

erreurs = y - preds_test
rmse_test = np.sqrt(np.mean(erreurs**2))
rmse_test

17.974434071502134